In [76]:
from reportlab.lib.pagesizes import landscape, letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
import pandas as pd

In [77]:
def split_list(lst, n):
    """Split list `lst` into chunks of length `n`."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [78]:
def generate_multi_table_pdf(data_file, output_pdf, max_cols_per_table=7):
    df = pd.read_csv(data_file)
    doc = SimpleDocTemplate(output_pdf, pagesize=landscape(letter))
    styles = getSampleStyleSheet()
    elements = list()

    elements.append(Paragraph("Automated Data Report", styles['Title']))
    elements.append(Spacer(1, 12))

    summary_text = df.describe().to_string()
    elements.append(Paragraph("Data Summary", styles['Heading2']))
    elements.append(Paragraph(summary_text.replace("\n", "<br />"), styles['BodyText']))
    elements.append(Spacer(1, 12))

    cols = df.columns.tolist()
    total_cols = len(cols)

    # Split columns into chunks to avoid overcrowding
    for col_chunk in split_list(cols, max_cols_per_table):
        # Prepare table header and data for this chunk
        table_data = [col_chunk] + df[col_chunk].astype(str).values.tolist()

        # Calculate column widths dynamically
        usable_width = landscape(letter)[0] - 2 * doc.leftMargin
        col_width = usable_width / len(col_chunk)
        col_widths = [col_width] * len(col_chunk)

        table = Table(table_data, colWidths=col_widths, repeatRows=1)

        style = TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, 0), 10),
            ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
            ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
            ('GRID', (0, 0), (-1, -1), 0.5, colors.black),
        ])

        table.setStyle(style)
        elements.append(table)
        elements.append(Spacer(1, 20))  # Space between tables

    doc.build(elements)
    print(f"Report generated and saved as {output_pdf}")

In [79]:
if __name__ == "__main__":
    generate_multi_table_pdf("sample.csv", "report.pdf")

Report generated and saved as report.pdf
